# AI Agents

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureOpenAI
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI
import requests
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import BaseTool, StructuredTool, tool

# Load environment variables from .env file
from pathlib import Path

env_path = r"C:\Users\vaalt\OneDrive\Desktop\Projects\Eventi speaker\Packt Bootcamp\code\Day 1\.env"
load_dotenv(dotenv_path=env_path, override=True)


# Access the environment variables
openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_chat_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")


# Initialize the Azure OpenAI model
model = AzureChatOpenAI(
    openai_api_version=openai_api_version,
    azure_deployment=azure_chat_deployment,
)

# Create a human message and invoke the model
message = HumanMessage(
    content="Translate this sentence from English to French. I love programming."
)
response = model.invoke([message])

# Print the response
print(response)


content="J'aime programmer." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 19, 'total_tokens': 24, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BiGQgKKmRInK8xx44Fm6MT5jQmrw0', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'f

In [2]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    api_key = openai_api_key,
    azure_deployment="text-embedding-3-large"
)

## SQL Agent

In [3]:
import pandas as pd
import sqlite3
import random

# ----------------------------
# 1. Product data
# ----------------------------
data = {
    "Product ID": [f"P{str(i).zfill(3)}" for i in range(1, 31)],
    "Product Name": [
        "Classic Piadina", "Vegetarian Piadina", "Ham & Cheese Piadina", 
        "Nutella Piadina", "Caprese Salad", "Tigelle Bread", 
        "Sun-Dried Tomatoes", "Parmesan Cheese", "Prosciutto Crudo", 
        "Balsamic Vinegar", "Truffle Piadina", "Spicy Salami Piadina", 
        "Vegan Piadina", "Chicken Caesar Salad", "Garlic Bread", 
        "Olive Tapenade", "Ricotta Cheese", "Mortadella", 
        "Pesto Sauce", "Marinated Artichokes", "Four Cheese Piadina", 
        "Grilled Eggplant", "Smoked Salmon Piadina", "Tuna Salad", 
        "Rosemary Focaccia", "Anchovy Spread", "Gorgonzola Cheese", 
        "Speck", "Fig Jam", "Lemon Olive Oil"
    ],
    "Category": [
        "Piadina", "Piadina", "Piadina", "Dessert", "Salad", "Bread", 
        "Condiment", "Cheese", "Meat", "Condiment", "Piadina", "Piadina", 
        "Piadina", "Salad", "Bread", "Condiment", "Cheese", "Meat", 
        "Condiment", "Condiment", "Piadina", "Vegetable", "Piadina", 
        "Salad", "Bread", "Condiment", "Cheese", "Meat", "Condiment", 
        "Condiment"
    ],
    "Price": [
        5.50, 6.00, 6.50, 4.00, 7.00, 3.50, 4.50, 8.00, 9.00, 6.00, 
        7.50, 6.75, 6.25, 8.50, 3.75, 5.25, 7.75, 8.50, 4.75, 5.50, 
        7.00, 4.25, 9.50, 7.25, 4.00, 5.75, 8.25, 9.25, 6.50, 5.00
    ],
    "Stock": [
        20, 15, 25, 10, 12, 30, 40, 18, 22, 25, 14, 16, 20, 10, 35, 
        28, 15, 12, 30, 25, 18, 22, 14, 10, 40, 20, 15, 12, 18, 25
    ],
    "Allergens": [
        "Gluten, Dairy", "Gluten, Dairy", "Gluten, Dairy, Pork", 
        "Gluten, Dairy, Nuts", "Dairy", "Gluten", "None", "Dairy", 
        "Pork", "None", "Gluten, Dairy, Truffle", "Gluten, Dairy, Pork", 
        "Gluten", "Dairy, Eggs", "Gluten", "None", "Dairy", "Pork", 
        "Nuts", "None", "Gluten, Dairy", "None", "Gluten, Dairy, Fish", 
        "Fish, Eggs", "Gluten", "Fish", "Dairy", "Pork", "None", "None"
    ],
    "Description": [
        "Traditional Italian flatbread filled with prosciutto and cheese.",
        "Flatbread filled with grilled vegetables and mozzarella.",
        "Flatbread filled with ham and melted cheese.",
        "Sweet flatbread filled with Nutella and crushed hazelnuts.",
        "Fresh salad with mozzarella, tomatoes, and basil.",
        "Soft round bread, perfect for pairing with spreads or cured meats.",
        "Rich and tangy sun-dried tomatoes, ideal for salads or toppings.",
        "Aged Parmesan cheese, perfect for grating over dishes.",
        "Thinly sliced cured ham, a classic Italian delicacy.",
        "Authentic balsamic vinegar from Modena, great for salads and marinades.",
        "Flatbread filled with truffle cream and cheese.",
        "Flatbread filled with spicy salami and mozzarella.",
        "Flatbread filled with vegan cheese and grilled vegetables.",
        "Crisp romaine lettuce with grilled chicken and Caesar dressing.",
        "Toasted bread with garlic and olive oil.",
        "Savory olive spread, perfect for appetizers.",
        "Creamy ricotta cheese, ideal for desserts or pasta.",
        "Thinly sliced mortadella, a flavorful Italian sausage.",
        "Fresh basil pesto sauce, great for pasta or sandwiches.",
        "Marinated artichokes, perfect for antipasti platters.",
        "Flatbread filled with a blend of four Italian cheeses.",
        "Grilled eggplant slices, seasoned with olive oil and herbs.",
        "Flatbread filled with smoked salmon and cream cheese.",
        "Fresh tuna salad with celery and mayonnaise.",
        "Soft focaccia bread infused with rosemary.",
        "Savory anchovy spread, ideal for bruschetta.",
        "Creamy Gorgonzola cheese, perfect for sauces or salads.",
        "Smoky cured speck, a northern Italian specialty.",
        "Sweet fig jam, great for pairing with cheese.",
        "Lemon-infused olive oil, perfect for dressings or marinades."
    ],
    "Calories": [
        350, 300, 400, 450, 250, 200, 50, 110, 150, 20, 380, 420, 310, 
        270, 180, 90, 120, 160, 80, 70, 400, 60, 450, 320, 220, 40, 
        130, 170, 100, 30
    ],
    "Vegetarian": [
        False, True, False, False, True, True, True, True, False, True, 
        True, False, True, False, True, True, True, False, True, True, 
        True, True, False, False, True, False, True, False, True, True
    ]
}

# ----------------------------
# 2. Supplier data
# ----------------------------
suppliers = {
    "Supplier ID": [f"S{str(i).zfill(3)}" for i in range(1, 11)],
    "Supplier Name": [
        "PiadaMaster Inc.", "Verde Farms", "Dolce Italia", 
        "Formaggi & Co.", "Salumi Rossi", "Mediterraneo Ltd.", 
        "Truffle Treasures", "Balsamico Tradizionale", 
        "Pane Fresco", "Olio di Sole"
    ],
    "Contact Email": [
        "contact@piadamaster.com", "info@verdefarms.com", 
        "orders@dolceitalia.it", "sales@formaggico.com", 
        "hello@salumirossi.it", "support@mediterraneo.com", 
        "info@truffletreasures.com", "shop@balsamicotradizionale.it", 
        "contact@panefresco.com", "service@oliodisole.it"
    ],
    "Country": [
        "Italy", "Italy", "Italy", "Italy", "Italy", 
        "Greece", "Italy", "Italy", "Italy", "Spain"
    ]
}

# ----------------------------
# 3. Create DataFrames
# ----------------------------
products_df = pd.DataFrame(data)
suppliers_df = pd.DataFrame(suppliers)

# ----------------------------
# 4. Assign a random Supplier ID to each product
# ----------------------------
supplier_ids = suppliers_df["Supplier ID"].tolist()
products_df["Supplier ID"] = [random.choice(supplier_ids) for _ in range(len(products_df))]

# ----------------------------
# 5. Save to SQLite
# ----------------------------
conn = sqlite3.connect("piadineria.db")

products_df.to_sql("products", conn, if_exists="replace", index=False)
suppliers_df.to_sql("suppliers", conn, if_exists="replace", index=False)

conn.close()


In [3]:
from langchain_community.utilities.sql_database import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///piadineria.db")

from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

sql_toolkit = SQLDatabaseToolkit(db=db, llm=model)
sql_toolkit.get_tools()

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000019ACB28E150>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000019ACB28E150>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000019ACB28E150>),
 QuerySQLCheckerTool(description='Use this tool to 

## Add to cart tool

In [4]:
# Define the tool to add an item to the cart
@tool
def add_to_cart(item_name: str, item_price: float) -> str:
    """Add an item to the cart."""
    url = 'http://localhost:3000/cart'  # Ensure this matches the JSON Server endpoint
    cart_item = {
        'name': item_name,
        'price': item_price
    }
    
    response = requests.post(url, json=cart_item)
    
    if response.status_code == 201:
        return f"Item '{item_name}' added to cart successfully."
    else:
        return f"Failed to add item to cart: {response.status_code} {response.text}"


# Retriever tool

In [6]:
%pip install faiss-cpu

   ---------------------------------------- 0.0/15.0 MB ? eta -:--:--
   ----- ---------------------------------- 2.1/15.0 MB 13.1 MB/s eta 0:00:01
   ------------ --------------------------- 4.7/15.0 MB 11.9 MB/s eta 0:00:01
   ------------------ --------------------- 7.1/15.0 MB 11.8 MB/s eta 0:00:01
   ------------------------- -------------- 9.7/15.0 MB 11.8 MB/s eta 0:00:01
   ---------------------------------------  14.9/15.0 MB 14.7 MB/s eta 0:00:01
   ---------------------------------------- 15.0/15.0 MB 14.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
import time
from langchain_text_splitters import CharacterTextSplitter

from langchain.document_loaders import PyPDFDirectoryLoader


index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

file_path = (
    "documents"
)
loader = PyPDFDirectoryLoader(file_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [9]:
from langchain_community.retrievers import AzureAISearchRetriever
vector_store.add_documents(documents=docs)

retriever = vector_store.as_retriever()

from langchain.tools.retriever import create_retriever_tool

rag_tool = create_retriever_tool(
    retriever,
    "document_search",
    """
    Search and return information restaurants health certificate and owner's history.
    """
)

## Prompt Design

In [10]:
# Define the prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """You are an AI assistant for a Piadineria Restaurant. 
            You help customers explore the menu and choose the best piadine or Italian specialties through friendly, interactive questions.
            When the user asks for product details (ingredients, allergens, vegetarian options, price, etc.), you can query the product database.

            Once the user is ready to order, ask if they'd like to add the selected item to their cart.
            If they confirm, add the item to the cart using your tools.

            When using a tool, respond only with the final result. For example:
            Human: Add Classic Piadina to the cart with price 5.50
            AI: Item 'Classic Piadina' added to cart successfully.
        """),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

## Running the Agent

In [12]:
import os
load_dotenv()
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = "askmamma"



In [13]:
from langchain.callbacks.tracers import LangChainTracer
from langsmith import Client

client = Client(
  api_key=os.getenv("LANGSMITH_API_KEY"),  # This can be retrieved from a secrets manager
  api_url=os.getenv("LANGSMITH_ENDPOINT"),  # Update appropriately for self-hosted installations or the EU region
)

tracer = LangChainTracer(client=client, project_name="askmamma")


In [14]:
# Setup the toolkit
toolkit = [rag_tool, add_to_cart, *sql_toolkit.get_tools()[:4]]

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
 
message_history = ChatMessageHistory()

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(model, toolkit, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=toolkit, verbose=True, return_intermediate_steps=True)

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)
 


# Interactive loop
while True:
    user_input = input("Type your question here (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break
    result = agent_with_chat_history.invoke(
        {"input": user_input},
        # This is needed because in most real world scenarios, a session id is needed
        # It isn't really used here because we are using a simple in memory ChatMessageHistory
        config={"configurable": {"session_id": "<foo>"}, "callbacks": [tracer]},
    )
    print(f'User: {user_input}')
    print(f'AI: {result['output']}')



> Entering new AgentExecutor chain...
Hello! Welcome to our Piadineria Restaurant. Are you interested in exploring our menu today?

> Finished chain.
User: hi
AI: Hello! Welcome to our Piadineria Restaurant. Are you interested in exploring our menu today?


> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with `{}`
responded: Let me check if we have ricotta cheese available in stock. Please hold on for a moment.

products, suppliers
Invoking: `sql_db_schema` with `{'table_names': 'products'}`



CREATE TABLE products (
	"Product ID" TEXT, 
	"Product Name" TEXT, 
	"Category" TEXT, 
	"Price" REAL, 
	"Stock" INTEGER, 
	"Allergens" TEXT, 
	"Description" TEXT, 
	"Calories" INTEGER, 
	"Vegetarian" INTEGER, 
	"Supplier ID" TEXT
)

/*
3 rows from products table:
Product ID	Product Name	Category	Price	Stock	Allergens	Description	Calories	Vegetarian	Supplier ID
P001	Classic Piadina	Piadina	5.5	20	Gluten, Dairy	Traditional Italian flatbread filled with prosciutto and cheese

In [15]:
result = agent_with_chat_history.invoke(
        {"input": "price of ricotta cheese?"},
        # This is needed because in most real world scenarios, a session id is needed
        # It isn't really used here because we are using a simple in memory ChatMessageHistory
        config={"configurable": {"session_id": "<foo>"}, "callbacks": [tracer]},
    )



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with `{}`


products, suppliers
Invoking: `sql_db_schema` with `{'table_names': 'products'}`



CREATE TABLE products (
	"Product ID" TEXT, 
	"Product Name" TEXT, 
	"Category" TEXT, 
	"Price" REAL, 
	"Stock" INTEGER, 
	"Allergens" TEXT, 
	"Description" TEXT, 
	"Calories" INTEGER, 
	"Vegetarian" INTEGER, 
	"Supplier ID" TEXT
)

/*
3 rows from products table:
Product ID	Product Name	Category	Price	Stock	Allergens	Description	Calories	Vegetarian	Supplier ID
P001	Classic Piadina	Piadina	5.5	20	Gluten, Dairy	Traditional Italian flatbread filled with prosciutto and cheese.	350	0	S009
P002	Vegetarian Piadina	Piadina	6.0	15	Gluten, Dairy	Flatbread filled with grilled vegetables and mozzarella.	300	1	S001
P003	Ham & Cheese Piadina	Piadina	6.5	25	Gluten, Dairy, Pork	Flatbread filled with ham and melted cheese.	400	0	S006
*/
Invoking: `sql_db_query_checker` with `{'query': 'SELECT Price FROM products WHERE "Product Name" = \'

In [16]:
result["intermediate_steps"]

[(ToolAgentAction(tool='sql_db_list_tables', tool_input={}, log='\nInvoking: `sql_db_list_tables` with `{}`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_SfQz7VflT3Pq7ld8kROLPxpr', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_ee1d74bde0'}, id='run--129641a4-08c0-41bf-8889-a3a700954603', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': 'call_SfQz7VflT3Pq7ld8kROLPxpr', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'sql_db_list_tables', 'args': '{}', 'id': 'call_SfQz7VflT3Pq7ld8kROLPxpr', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_SfQz7VflT3Pq7ld8kROLPxpr'),
  'products, suppliers'),
 (ToolAgentAction(tool='sql_db_schema', tool_input={'table_names': 'products'}, log="\nInvoking: `sql_db_schema` with `{'table_names': 'products'}`\n\n\n", message_

In [19]:
intermediate_steps = result["intermediate_steps"]

for i, (action, observation) in enumerate(intermediate_steps, 1):
    print(f"\n🔍 Step {i}")
    print(f"🛠️ Tool Used: {action.tool}")
    print(f"🧾 Tool Input: {action.tool_input}")
    
    if action.log.strip():
        print(f"🧠 Log:\n{action.log.strip()}")

    print("📨 Message Log Summary:")
    for msg in action.message_log:
        tool_calls = msg.additional_kwargs.get("tool_calls", [])
        for call in tool_calls:
            func = call.get("function", {})
            func_name = func.get("name", "N/A")
            func_args = func.get("arguments", "N/A")
            print(f"   🔧 Function: `{func_name}` | Args: `{func_args}`")
    
    print("📤 Observation:")
    obs_str = str(observation).strip()
    if len(obs_str) > 800:
        print(obs_str[:800] + "...\n📝 (truncated for brevity)")
    else:
        print(obs_str)

    print("⏱️ " + "─" * 60)



🔍 Step 1
🛠️ Tool Used: sql_db_list_tables
🧾 Tool Input: {}
🧠 Log:
Invoking: `sql_db_list_tables` with `{}`
📨 Message Log Summary:
   🔧 Function: `sql_db_list_tables` | Args: `{}`
📤 Observation:
products, suppliers
⏱️ ────────────────────────────────────────────────────────────

🔍 Step 2
🛠️ Tool Used: sql_db_schema
🧾 Tool Input: {'table_names': 'products'}
🧠 Log:
Invoking: `sql_db_schema` with `{'table_names': 'products'}`
📨 Message Log Summary:
   🔧 Function: `sql_db_schema` | Args: `{"table_names":"products"}`
📤 Observation:
CREATE TABLE products (
	"Product ID" TEXT, 
	"Product Name" TEXT, 
	"Category" TEXT, 
	"Price" REAL, 
	"Stock" INTEGER, 
	"Allergens" TEXT, 
	"Description" TEXT, 
	"Calories" INTEGER, 
	"Vegetarian" INTEGER, 
	"Supplier ID" TEXT
)

/*
3 rows from products table:
Product ID	Product Name	Category	Price	Stock	Allergens	Description	Calories	Vegetarian	Supplier ID
P001	Classic Piadina	Piadina	5.5	20	Gluten, Dairy	Traditional Italian flatbread filled with prosciutto a

# (Optional) Agent output evaluation

In [20]:
from langsmith import Client

client = Client(
  api_key=os.getenv("LANGSMITH_API_KEY"),  # This can be retrieved from a secrets manager
  api_url=os.getenv("LANGSMITH_ENDPOINT"),  # Update appropriately for self-hosted installations or the EU region
)
from openai import AzureOpenAI

aoai_client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = os.getenv("AZURE_OPENAI_API_VERSION")
    )

from langsmith import wrappers





In [18]:
dataset = client.create_dataset(
    dataset_name="QA Askmamma", description="A sample dataset in LangSmith."
)

# Create examples
examples = [
    {
        "inputs": {"question": "What is Piadina?"},
        "outputs": {"answer": "A traditional Italian flatbread, typically made with wheat flour, water, and salt."},
    },
    {
        "inputs": {"question": "What is the tradition of Piadina?"},
        "outputs": {"answer": "Piadina is a traditional Italian flatbread that originated in the Romagna region. It is typically filled with various ingredients and served warm."},
    },
]

# Add examples to the dataset
client.create_examples(dataset_id=dataset.id, examples=examples)

{'example_ids': ['11135b9b-5505-46a0-b8e8-d7a656c10722',
  '4d5f4ad5-c502-40f1-b3fa-13e9ac7829ff'],
 'count': 2}

In [21]:
openai_client = wrappers.wrap_openai(aoai_client)
def target(inputs: dict) -> dict:
    response = agent_with_chat_history.invoke(
        {"input": inputs['question']},
        # This is needed because in most real world scenarios, a session id is needed
        # It isn't really used here because we are using a simple in memory ChatMessageHistory
        config={"configurable": {"session_id": "<foo>"}, "callbacks": [tracer]},
    )
    return { "response": response['output'] }

In [22]:
eval_instructions = "You are an expert professor specialized in grading students' answers to questions."

def correctness(inputs: dict, outputs: dict, reference_outputs: dict) -> bool:
    user_content = f"""You are grading the following question:
{inputs['question']}
Here is the real answer:
{reference_outputs['answer']}
You are grading the following predicted answer:
{outputs['response']}
Respond with a score of 1-5, where 1 is the worst and 5 is the best. Your answer ONLY contains the score.
"""
    response = openai_client.chat.completions.create(
        model="gpt-4o",
        temperature=0,
        messages=[
            {"role": "system", "content": eval_instructions},
            {"role": "user", "content": user_content},
        ],
    ).choices[0].message.content
    return response.strip()

In [23]:
# After running the evaluation, a link will be provided to view the results in langsmith
experiment_results = client.evaluate(
    target,
    data="QA Askmamma",
    evaluators=[
        correctness,
        # can add multiple evaluators here
    ],
    experiment_prefix="first-eval-in-langsmith",
    max_concurrency=2,
)

View the evaluation results for experiment: 'first-eval-in-langsmith-ddbc0ca1' at:
https://smith.langchain.com/o/77bc0d8c-7dec-4b89-8656-81cffecbaeab/datasets/5d92c440-166d-49a3-a202-ba8acb3cbaca/compare?selectedSessions=1eb39c54-31eb-4e5b-802d-68595d479b40




0it [00:00, ?it/s]



> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...
Piadina is a traditional Italian flatbread originating from the region of Emilia-Romagna. It's typically made from a simple dough consisting of flour, water, salt, and lard (or olive oil for a vegetarian version). Piadina is cooked on a hot griddle or pan until golden brown and slightly crispy on the outside, while remaining soft and chewy inside.

It's often used as a wrap or folded sandwich, filled with various savory ingredients such as cheese, cured meats, vegetables, and spreads. Piadina is popular as a quick and delicious street food snack or meal. Would you like to know more about our piadina options or explore other menu items?

> Finished chain.
Piadina is a traditional Italian flatbread originating from the Emilia-Romagna region in Northern Italy. It's known for its simple ingredients and delicious versatility. Here's a bit more about its tradition:

1. **Historical Roots:** Piadina has ancient or

# (Optional) Agent tool evaluation

In [24]:
import uuid


questions = [
    (
        "Do you have ricotta cheese in stock",
        {
            "reference": "Yes we do have ricotta cheese in stock.",
            "expected_steps": ["sql_db_list_tables", "sql_db_schema", "sql_db_query_checker", "sql_db_query"],
        },
    ),
    (
        "hi",
        {
            "reference": "Hello, how can I assist you?",
            "expected_steps": [],  # Expect a direct response
        },
    ),
    (
        "Can you add ricotta cheese to the cart with price 5.50?",
        {
            "reference": "The item 'ricotta cheese' has been added to the cart.",
            "expected_steps": ["add_to_cart"],
        },
    ),
    (
        "Do you have food safety certificate?",
        {
            "reference": "Yes, we have a food safety certificate.",
            "expected_steps": ["document_search"],
        },
    ),
]

uid = uuid.uuid4()
dataset_name = f"Agent Tool Eval Example {uid}"
ds = client.create_dataset(
    dataset_name=dataset_name,
    description="An example agent evals dataset using search and calendar checks.",
)
client.create_examples(
    inputs=[{"input": q[0]} for q in questions],
    outputs=[q[1] for q in questions],
    dataset_id=ds.id,
)

{'example_ids': ['5756a68c-b57f-45e5-bf69-9c1dbd955b96',
  'cd347d78-1de0-46be-b0bb-ad4ef6a383e2',
  '2182279e-0964-40b2-9a91-f4d2cb0260e0',
  'cf79804f-ae1f-4e6c-9be2-f828d7990abb'],
 'count': 4}

In [25]:
from typing import Optional

from langsmith.schemas import Example, Run


def intermediate_step_correctness(run: Run, example: Optional[Example] = None) -> dict:
    if run.outputs is None:
        raise ValueError("Run outputs cannot be None")
    # This is the output of each run
    intermediate_steps = run.outputs.get("intermediate_steps") or []
    # Since we are comparing to the tool names, we now need to get that
    # Intermediate steps is a Tuple[AgentAction, Any]
    # The first element is the action taken
    # The second element is the observation from taking that action
    trajectory = [action.tool for action, _ in intermediate_steps]
    # This is what we uploaded to the dataset
    expected_trajectory = example.outputs["expected_steps"]
    # Just score it based on whether it is correct or not
    score = int(trajectory == expected_trajectory)
    return {"key": "Intermediate steps correctness", "score": score}

In [26]:
def agent(inputs: dict):

    return agent_with_chat_history.invoke(inputs, config={"configurable": {"session_id": "<foo>"}})

In [27]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate


# We now need to specify this because we have multiple outputs in our dataset
def prepare_data(run: Run, example: Example) -> dict:
    return {
        "input": example.inputs["input"],
        "prediction": run.outputs["output"],
        "reference": example.outputs["reference"],
    }


# Measures whether a QA response is "Correct", based on a reference answer
qa_evaluator = LangChainStringEvaluator("qa", prepare_data=prepare_data, config={"llm": model})
chain_results = evaluate(
    agent,
    data=dataset_name,
    evaluators=[intermediate_step_correctness, qa_evaluator],
    experiment_prefix="Agent Eval Example",
    max_concurrency=1,
)

View the evaluation results for experiment: 'Agent Eval Example-ce6e4957' at:
https://smith.langchain.com/o/77bc0d8c-7dec-4b89-8656-81cffecbaeab/datasets/35a149f2-e5b6-41a3-b3cd-6a8d37e184ac/compare?selectedSessions=f628e8e5-f88b-4edd-bb02-3f4991fdf5da




0it [00:00, ?it/s]



> Entering new AgentExecutor chain...

Invoking: `add_to_cart` with `{'item_name': 'Ricotta Cheese', 'item_price': 5.5}`


Item 'Ricotta Cheese' added to cart successfully.Ricotta Cheese has been successfully added to your cart at the price of $5.50. Would you like to explore more menu items or proceed to checkout?

> Finished chain.


> Entering new AgentExecutor chain...
Yes, we have ricotta cheese in stock with 15 units available. Feel free to add more to your cart if you'd like! Let me know if there's anything else you need from the menu.

> Finished chain.


> Entering new AgentExecutor chain...
Hello! Welcome back to our Piadineria Restaurant. How can I assist you today? Are you interested in exploring our menu or have any questions?

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `document_search` with `{'query': 'food safety certificate'}`


Piadineria Hygiene & Food Safety Compliance Report
Page 2: HACCP Certification
Hazard Analysis and Critical Contro